In [3]:
import pandas as pd
import json
import sys
import datetime
import shutil
import os
import datetime

from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from utils import *
from gpt_functions import *
from edgelist import *

load_dotenv() # Cargar las variables de entorno desde el archivo .env

False

In [57]:
# Initializing OpenAI client - see https://platform.openai.com/docs/quickstart?context=python
client = OpenAI()

In [11]:
df = pd.read_csv('test.csv')
len(df)


868

In [72]:
df = df.iloc[0:10000, :]
len(df)

10000

In [ ]:
df = df.iloc[10000:18264 , :]
len(df)

In [59]:
prompt_path = '../prompts/prompt.txt'

prompt = load_prompt(prompt_path)

In [38]:
lista = "['Candelaria Acevedo', 'Eric Aedo', 'Yovana Ahumada', 'Jorge Alessandri', 'René Alinco', 'Jaime Araya', 'Cristián Araya', 'Mónica Arce', 'Roberto Arroyo', 'Danisa Astudillo', 'Chiara Barchiesi', 'Boris Barrera', 'Héctor Barría', 'Miguel Ángel Becker', 'Francisca Bello', 'Juan Carlos Beltrán', 'Gustavo Benavente', 'Bernardo Berger', 'Alejandro Bernales', 'Carlos Bianchi', 'Sergio Bobadilla', 'Fernando Bórquez', 'Ana María Bravo', 'Marta Bravo', 'Jorge Brito', 'Félix Bugueño', 'Mercedes Bulnes', 'Miguel Ángel Calisto', 'Felipe Camaño', 'Karol Cariola', 'Álvaro Carter', 'Nathalie Castillo', 'José Miguel Castro', 'Andrés Celis', 'Daniella Cicardini', 'Sofía Cid', 'Ricardo Cifuentes', 'Juan Antonio Coloma', 'Sara Concha', 'María Luisa Cordero', 'Eduardo Cornejo', 'Luis Cuello', 'Gonzalo De la Carrera', 'Tomás de Rementería', 'Catalina del Real', 'Viviana Delgado', 'Felipe Donoso', 'Jorge Durán', 'Eduardo Durán', 'Camila Flores', 'Lorena Fries', 'Juan Fuenzalida', 'Ana María Gazmuri', 'Andrés Giordano', 'Félix González', 'Marta González', 'Mauro González', 'Jorge Guzmán', 'Carmen Hertz', 'Tomás Hirsch', 'Diego Ibáñez', 'Marcos Ilabaca', 'Juan Irarrázaval', 'Pamela Jiles', 'Andrés Jouannet', 'Harry Jürgensen', 'Johannes Kaiser', 'Cristian Labbé', 'Paula Labra', 'Tomás Lagomarsino', 'Joaquín Lavín', 'Henry Leal', 'Enrique Lee', 'Raúl Leiva', 'Daniel Lilayu', 'Andrés Longton', 'Luis Malla', 'Daniel Manouchehri', 'Cristóbal Martínez', 'Carolina Marzán', 'Christian Matheson', 'Karen Medina', 'Cosme Mellado', 'Miguel Mellado', 'Daniel Melo', 'José Carlos Meza', 'Vlado Mirosevic', 'Claudia Mix', 'Helia Molina', 'Javiera Morales', 'Carla Morales', 'Cristhian Moreira', 'Benjamín Moreno', 'Jaime Mulet', 'Francesca Muñoz', 'Camila Musante', 'Jaime Naranjo', 'Gloria Naveillan', 'Emilia Nuyado', 'Ericka Ñanco', 'Mauricio Ojeda', 'Erika Olivera', 'Maite Orsini', 'Ximena Ossandón', 'Rubén Darío Oyarzo', 'Hernán Palma', 'Marlene Pérez', 'Joanna Pérez', 'Catalina Pérez', 'Víctor Alejandro Pino', 'Lorena Pizarro', 'Alejandra Placencia', 'Francisco Pulgar', 'Guillermo Ramírez', 'Matías Ramírez', 'Marcia Raphael', 'Jorge Rathgeb', 'Hugo Rey', 'Marcela Riquelme', 'Gaspar Rivas', 'Camila Rojas', 'Agustín Romero', 'Leonidas Romero', 'Natalia Romero', 'Patricio Rosas', 'Jaime Sáez', 'Jorge Saffirio', 'Clara Sagardia', 'Luis Sánchez', 'Juan Santana', 'Marisela Santibáñez', 'Frank Sauerbaum', 'Diego Schalper', 'Emilia Schneider', 'Stephan Schubert', 'Alexis Sepúlveda', 'Daniela Serrano', 'Leonardo Soto', 'Raúl Soto', 'Marco Antonio Sulantay', 'Cristián Tapia', 'Hotuiti Teao', 'Carolina Tello', 'Renzo Trisotti', 'Héctor Ulloa', 'Francisco Undurraga', 'Alberto Undurraga', 'Cristóbal Urruticoechea', 'Consuelo Veloso', 'Nelson Venegas', 'Sebastián Videla', 'Gastón Von Mühlenbrock', 'Flor Weisse', 'Gonzalo Winter', 'Gael Yeomans'] "

In [60]:
prompt = prompt

In [63]:
print(prompt)

Your goal is to identify all the individuals mentioned in the document and provide information about each person as a structured object. 
You should also identify relationships between the individuals. Consider only people as individuals.
The answer should contain "nodes" and "edges":
nodes: List of dictionaries with keys "id" (unique number identifier), "label" (name of person). Only people should be included
edges: List of dictionaries with "from" (initiator), "to" (receiver), "label" (1-word summary of the relation between the individuals in English) and "sentiment" of the relationship (positive, negative, or neutral).

Ensure that the 'id' field in 'nodes' matches the 'from' and 'to' fields in 'edges' to accurately represent relationships between individuals.
Generate a new JSON object containing the response:

{
    "nodes": [
            "id": integer // ID of one individual
            "label": string //  First name and Last name of the individual
      ],
    "edges": [ // cont

In [65]:
# Crear una lista de tareas en formato JSON
tasks = []

# Itera sobre el DataFrame
for index, row in df.iterrows():
    
    description = row['text']
    
    # Crear el objeto JSON para la tarea
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "temperature": 0.1,
            "response_format": { 
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",
                    "content": description
                }
            ],
        }
    }
    
    # Añadir la tarea a la lista
    tasks.append(task)

# Ahora 'tasks' contiene todos los objetos JSON para las solicitudes API

In [66]:
# Creating the file

file_name = "../data/batch_tasks_text2graph.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [67]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)
print(batch_file)

FileObject(id='file-ZRBnDEl1j8YySeJLt4kLpV6s', bytes=2235615, created_at=1724882733, filename='batch_tasks_text2graph.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [68]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [45]:
id = "batch_TffA7DVhmgerXOHjsuiSUvEV"
batch_job = client.batches.retrieve(id)
print(batch_job) #  checking until the status is 'completed'.

Batch(id='batch_TffA7DVhmgerXOHjsuiSUvEV', completion_window='24h', created_at=1724880708, endpoint='/v1/chat/completions', input_file_id='file-20ctEr2eQ1gPK7iUQ9jV6NNi', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1724881026, error_file_id=None, errors=None, expired_at=None, expires_at=1724967108, failed_at=None, finalizing_at=1724880992, in_progress_at=1724880709, metadata=None, output_file_id='file-coOd2URyOz37Fa3YLetfImFA', request_counts=BatchRequestCounts(completed=868, failed=0, total=868))


# Recolectar los resultados

In [5]:
jsonl_file_path = "../Batch_results_V0/batch_brCSiI7grMknsnbBoa2eF0O9_output.jsonl" # Resultados en json
csv_file_path= "../Batch_results_V0/batch_results_2.csv"

In [6]:
def read_jsonl(file_path):
    with open(file_path, 'r') as file:
        # Leer todas las líneas del archivo
        lines = file.readlines()
        
        # Convertir cada línea en un diccionario Python
        json_lines = [json.loads(line) for line in lines]
        
    return json_lines
def process_json_data(json_lines):
    # Crear un DataFrame a partir de la lista de diccionarios
    df = pd.json_normalize(json_lines)
    
    return df
# Convertir a CSV
def convert_to_csv(json_lines, csv_path):
    df = process_json_data(json_lines)
    df.to_csv(csv_path, index=False)

In [7]:
json_lines = read_jsonl(jsonl_file_path)
convert_to_csv(json_lines, csv_file_path)

print(f"Archivo CSV guardado en: {csv_file_path}")

Archivo CSV guardado en: ../Batch_results_V0/batch_results_2.csv


In [8]:
batch_df = pd.read_csv(csv_file_path)

In [9]:
batch_df

,id,custom_id,error,response.status_code,response.request_id,response.body.id,response.body.object,response.body.created,response.body.model,response.body.choices,response.body.usage.prompt_tokens,response.body.usage.completion_tokens,response.body.usage.total_tokens,response.body.system_fingerprint
0,batch_req_zLB1vy7QNVkmyr6L1V2TpWYb,task-0,NaN,200,54c496aa7735ec8c303528ad3b16acd2,chatcmpl-A1L5EZZC3O5O19vYDInx2nWRcaqhc,chat.completion,1724882736,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",531,236,767,fp_157b3831f5
1,batch_req_dHdyDw8d4OuC4W7Cf65IqqJ8,task-1,NaN,200,93fd79d6116921553667b1415c565519,chatcmpl-A1L5EZ0lBeS6tiPNc1HOj3DWYlydi,chat.completion,1724882736,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",504,92,596,fp_157b3831f5
2,batch_req_PFm0qnxQDICY4jTvXYvePt9D,task-2,NaN,200,23eaed5af1d3e317046447d3d1a371cd,chatcmpl-A1L5FSaR5MEXlWVd7Rtwudc45Z7yB,chat.completion,1724882737,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",509,92,601,fp_157b3831f5
3,batch_req_CzYnB2SRswdNH5fDwSZEGqgG,task-3,NaN,200,8c6e5bb4302b219754e2cd0bc33b44ae,chatcmpl-A1L5FPFuZvESZY5W5IO3K65SnLoah,chat.completion,1724882737,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",528,310,838,fp_157b3831f5
4,batch_req_nFuUa2BaysGahBPHENJpRANu,task-4,NaN,200,861f5de9da7f75aefa04dd616a1307e2,chatcmpl-A1L5F3zY7OWYS7s1Mmt5df5tq7ZUw,chat.completion,1724882737,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",511,170,681,fp_cb7cc8e106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,batch_req_K39J63zvslFc4LkIAY9fYhGF,task-863,NaN,200,99d7dc34162b4955a573f2ecb02117fa,chatcmpl-A1L750ZOFqBCkOy80E66wEdwf2szm,chat.completion,1724882851,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",519,144,663,fp_157b3831f5
864,batch_req_Vd90pt6nQdh6yqewZMyqmJ9S,task-864,NaN,200,f6db24867d299defa1e65bf1d3ecb056,chatcmpl-A1L754izpVT9iCrIQOwITKtVBLQ8N,chat.completion,1724882851,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",514,93,607,fp_157b3831f5
865,batch_req_YOUVnx6Hax4u5UqIKKxDwWKf,task-865,NaN,200,96091be5c7379504ffd448b0517841a2,chatcmpl-A1L76fIgppKd9mrApHbxpEzuOKciu,chat.completion,1724882852,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",542,167,709,fp_157b3831f5
866,batch_req_OQeKHjFCWY1GDIdEbMwOmwT3,task-866,NaN,200,c39feabcdd12699b428df7f34da19316,chatcmpl-A1L75rvLVxCTBCg54hNkJbgm025Ej,chat.completion,1724882851,gpt-4o-2024-05-13,"[{'index': 0, 'message': {'role': 'assistant',...",520,1444,1964,fp_157b3831f5


# Reading results

In [12]:
import pandas as pd

results_list = []

json_dict = {res['custom_id']: res for res in json_lines}

for res in json_lines:
    task_id = res['custom_id']
    # Obtener el índice del task_id
    index = int(task_id.split('-')[-1])
    result = res['response']['body']['choices'][0]['message']['content']
    
    # Almacenar el resultado, el índice y la línea JSON completa en la lista
    results_list.append({
        'index': index,
        'result': result,
        'output_batch': json_dict.get(task_id)  # Agregar la línea JSON completa
    })

# Convertir la lista en un DataFrame
results_df = pd.DataFrame(results_list)

# Combinar con el DataFrame original usando el índice para preservar el orden original
df_results = df.merge(results_df, left_index=True, right_on='index', how='left')

# Guardar el DataFrame resultante en un archivo CSV
df_results.to_csv('../Batch_results_V0/resultados_batch_merge_input.csv', index=False)


In [13]:
df_batch_merge_input = pd.read_csv('../Batch_results_V0/resultados_batch_merge_input.csv')

In [14]:
df_batch_merge_input

,Unnamed: 0.1,Unnamed: 0,text,response,index,result,output_batch
0,0,694,"La ministra vocera de Gobierno, Camila Vallejo...","{""nodes"": [{""id"": 1, ""label"": ""Camila Vallejo""...",0,"{\n ""nodes"": [\n {\n ""id""...","{'id': 'batch_req_zLB1vy7QNVkmyr6L1V2TpWYb', '..."
1,1,695,En Tu Día conversamos con la diputada Pamela J...,"{""nodes"": [{""id"": 1, ""label"": ""Pamela Jiles"", ...",1,"{\n ""nodes"": [\n {\n ""id""...","{'id': 'batch_req_dHdyDw8d4OuC4W7Cf65IqqJ8', '..."
2,2,696,Con 11 votos a favor y 2 en contra de los parl...,"{""nodes"": [{""id"": 1, ""label"": ""Pamela Jiles"", ...",2,"{\n ""nodes"": [\n {\n ""id""...","{'id': 'batch_req_PFm0qnxQDICY4jTvXYvePt9D', '..."
3,3,697,"Durante la jornada de este lunes, la Cámara de...","{""nodes"": [{""id"": 1, ""label"": ""Cristián Labbé""...",3,"{\n ""nodes"": [\n {\n ""id""...","{'id': 'batch_req_CzYnB2SRswdNH5fDwSZEGqgG', '..."
4,4,698,El candidato a La Moneda del Frente Social Cri...,"{""nodes"": [{""id"": 1, ""label"": ""José Antonio Ka...",4,"{\n ""nodes"": [\n {\n ""id""...","{'id': 'batch_req_nFuUa2BaysGahBPHENJpRANu', '..."
...,...,...,...,...,...,...,...
863,863,689,El pasado lunes se dio a conocer que la diputa...,"{""nodes"": [{""id"": 1, ""label"": ""Camila Flores"",...",863,"{\n ""nodes"": [\n {\n ""id""...","{'id': 'batch_req_K39J63zvslFc4LkIAY9fYhGF', '..."
864,864,690,"""Estos personajes que trabajan 4 años y de por...","{""nodes"": [{""id"": 1, ""label"": ""Leonidas Romero...",864,"{\n ""nodes"": [\n {\n ""id""...","{'id': 'batch_req_Vd90pt6nQdh6yqewZMyqmJ9S', '..."
865,865,691,El congresista estuvo invitado en el matinal M...,"{""nodes"": [{""id"": 1, ""label"": ""Rojo Edwards"", ...",865,"{\n ""nodes"": [\n {\n ""id""...","{'id': 'batch_req_YOUVnx6Hax4u5UqIKKxDwWKf', '..."
866,866,692,La Cámara de Diputadas y Diputados puso una tr...,"{""nodes"": [{""id"": 1, ""label"": ""Marco Sulantay""...",866,"{\n ""nodes"": [\n {""id"": 1, ""label"": ...","{'id': 'batch_req_OQeKHjFCWY1GDIdEbMwOmwT3', '..."


In [15]:
print(df_batch_merge_input['result'].iloc[0])

{
    "nodes": [
        {
            "id": 1,
            "label": "camila vallejo"
        },
        {
            "id": 2,
            "label": "gabriel boric"
        },
        {
            "id": 3,
            "label": "claudia mix"
        },
        {
            "id": 4,
            "label": "nathalie castillo"
        }
    ],
    "edges": [
        {
            "from": 1,
            "to": 2,
            "label": "indicated",
            "sentiment": "neutral"
        },
        {
            "from": 3,
            "to": 2,
            "label": "commented",
            "sentiment": "neutral"
        },
        {
            "from": 4,
            "to": 2,
            "label": "declared",
            "sentiment": "neutral"
        },
        {
            "from": 1,
            "to": 3,
            "label": "responded",
            "sentiment": "neutral"
        }
    ]
}


In [ ]:
# df_combined = parse_string_to_json(df_batch_merge_input, 'result')

In [ ]:
# edgelist_df = edgelist_version(df_gpt, 'inference_parsed')


# Agregar date a cada edge

In [55]:
def add_date_to_edges(result_str, date):
    # Convertir la cadena JSON a un diccionario
    result_dict = json.loads(result_str)
    
    # Agregar la fecha a cada edge
    for edge in result_dict['edges']:
        edge['date'] = date
    
    # Convertir el diccionario de nuevo a una cadena JSON
    return json.dumps(result_dict)


In [56]:
df_nuevo = df_batch_merge_input.copy()  # Hacer una copia del DataFrame original


In [57]:
df_nuevo['result'] = df_nuevo.apply(lambda row: add_date_to_edges(row['result'], row['date']), axis=1)


In [59]:
result_str = df_nuevo['result'].iloc[0]

result_dict = json.loads(result_str)

# Imprimir el JSON de manera legible
print(json.dumps(result_dict, indent=4, ensure_ascii=False))

{
    "nodes": [
        {
            "id": 1,
            "label": "victor hugo rice",
            "summary": "el alcalde de san nicolás que decretó el uso obligatorio de mascarillas en la comuna."
        },
        {
            "id": 2,
            "label": "cesar riquelme",
            "summary": "el seremi de educación de ñuble que recibió la solicitud de suspensión de clases."
        },
        {
            "id": 3,
            "label": "sergio hernandez",
            "summary": "el jefe administrativo del departamento de educación municipal que comentó sobre la suspensión de clases."
        },
        {
            "id": 4,
            "label": "victor reyes",
            "summary": "el director del liceo bicentenario polivalente de san nicolás que habló sobre las medidas para jardines infantiles."
        }
    ],
    "edges": [
        {
            "from": 1,
            "to": 3,
            "label": "collaboration",
            "sentiment": "positive",
            "date

In [17]:
df_batch_merge_input.to_csv("../Batch_results_V0/output_GPT_lista_cerrada_test.csv")